<a href="https://colab.research.google.com/github/ayush572/Fake_News_Classification_LSTM/blob/main/Fake_News_Classification_Kaggle_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install opendatasets

In [25]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/c/fake-news/data#/")


Skipping, found downloaded files in "./fake-news" (use force=True to force download)


In [26]:
import pandas as pd
df = pd.read_csv('fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [27]:
# dropping nan values
df = df.dropna()

In [28]:
# now removing the label as label is the output, so removing it in the X
X = df.drop('label', axis = 1)

In [29]:
# y contains only the output of the same
Y = df['label']

In [30]:
print(X.shape)
Y.shape

(18285, 4)


(18285,)

In [31]:
import tensorflow as tf

In [32]:
# importing those things that are necessary to use LSTM
from tensorflow.keras.layers import Embedding # to convert the one-hot encoded vectors to featurized vectors
from tensorflow.keras.preprocessing.sequence import pad_sequences # helps to make the sentences of equal length
from tensorflow.keras.models import Sequential # to create a sequential model
from tensorflow.keras.preprocessing.text import one_hot # to convert the words to one_hot encoded vectors
from tensorflow.keras.layers import LSTM, Dense # using of LSTM and Dense layers
# Dense layer is being used as this is a classification problem, so that's why we require the same

In [33]:
# voca size - it defines like say within how many words our sentences will like
vocab_size=5000

In [34]:
messages = X.copy()

In [35]:
messages.reset_index(inplace = True) # as we have dropped the nan values, so, it leaves gaps in the rows. So, to counter that, we do reset_index

In [36]:
import nltk
import re
from nltk.corpus import stopwords

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
# Dataset preprocessing
from nltk.stem.porter import PorterStemmer # for stemming
ps = PorterStemmer()
corpus = []
for i in range(len(messages)): # len(messages) -> length of X
  # print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # means, replacing all the characters, apart from a-z and A-Z, with '' blank space
  review = review.lower()
  review = review.split() # splitting - default delimiter white space
  # now review will be having the list of words insisde that specific sentence

  stemmed_words = []
  for word in review:
    if word not in stopwords.words('english'):
      word = ps.stem(word)
      stemmed_words.append(word)
  review = stemmed_words
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [45]:
corpus

['house dem aide we didn t even see comey s letter until jason chaffetz tweeted it',
 'flynn hillary clinton big woman on campus breitbart',
 'why the truth might get you fired',
 'civilians killed in single us airstrike have been identified',
 'iranian woman jailed for fictional unpublished story about woman stoned to death for adultery',
 'jackie mason hollywood would love trump if he bombed north korea over lack of trans bathrooms exclusive video breitbart',
 'beno t hamon wins french socialist party s presidential nomination the new york times',
 'a back channel plan for ukraine and russia courtesy of trump associates the new york times',
 'obama s organizing for action partners with soros linked indivisible to disrupt trump s agenda',
 'bbc comedy sketch real housewives of isis causes outrage',
 'russian researchers discover secret nazi military base treasure hunter in the arctic photos',
 'us officials see no link between trump and russia',
 're yes there are paid government trol

In [46]:
onehot_repr = [one_hot(words, vocab_size) for words in corpus] # traversing through all the words in corpus
onehot_repr

[[4775,
  3362,
  2599,
  1166,
  2037,
  3133,
  333,
  4991,
  3318,
  4354,
  4568,
  1244,
  2136,
  3792,
  1275,
  3943],
 [2462, 1043, 2512, 1900, 4383, 3750, 1270, 1732],
 [4022, 3311, 1319, 2006, 1473, 3547, 3466],
 [88, 3631, 3062, 3615, 3055, 3137, 3344, 3580, 4181],
 [1227,
  4383,
  166,
  1868,
  2999,
  2942,
  2893,
  4012,
  4383,
  2234,
  2591,
  3998,
  1868,
  3679],
 [3453,
  119,
  4883,
  4360,
  3973,
  3376,
  2388,
  3693,
  1516,
  3830,
  3728,
  3962,
  156,
  1975,
  2644,
  1390,
  2059,
  1538,
  1732],
 [1514,
  3133,
  3231,
  666,
  3709,
  3413,
  3954,
  4354,
  315,
  168,
  3311,
  4202,
  1808,
  3652],
 [995,
  3762,
  4378,
  3060,
  1868,
  3464,
  4868,
  777,
  3668,
  1975,
  3376,
  4036,
  3311,
  4202,
  1808,
  3652],
 [401,
  4354,
  17,
  1868,
  782,
  3445,
  4886,
  2461,
  2862,
  1255,
  2591,
  1132,
  3376,
  4354,
  1834],
 [4247, 4244, 3966, 780, 3614, 1975, 1536, 1139, 4402],
 [3174, 4359, 446, 3354, 3489, 4214, 669, 3098, 

In [51]:
# now as all the sentences length were different, so we use pad_sequences to convert the same to sentences of equal length
max_length_list = max(onehot_repr, key=len)
max_length = len(max_length_list)

print("List with maximum length:", max_length_list)
print("Maximum length:", max_length)


List with maximum length: [4943, 1808, 4548, 3750, 3665, 3311, 2736, 940, 4814, 1023, 4798, 75, 858, 1350, 2558, 1975, 995, 1978, 4631, 153, 401, 904, 447, 1515, 4273, 3744, 2591, 1781, 2570, 2558, 2034, 2880, 3611, 3820, 613, 3099, 2461, 1043, 4354, 4770, 1975, 1072, 3311, 3055, 3825, 4234, 2594, 4149, 3311, 3936, 3311, 2924, 4252, 2933, 4841, 4868, 2280, 4868, 3311, 1394, 1219, 4234, 4899, 1900, 3311, 1618, 875, 4234, 2523]
Maximum length: 69


In [52]:
# Pad sequences to the maximum length
padded_sequences = pad_sequences(onehot_repr, maxlen=max_length, padding='pre')

In [53]:
print(padded_sequences)

[[   0    0    0 ... 3792 1275 3943]
 [   0    0    0 ... 3750 1270 1732]
 [   0    0    0 ... 1473 3547 3466]
 ...
 [   0    0    0 ... 4202 1808 3652]
 [   0    0    0 ... 2424 3062 3081]
 [   0    0    0 ... 3311 4562 3309]]


In [54]:
padded_sequences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 4775, 3362,
       2599, 1166, 2037, 3133,  333, 4991, 3318, 4354, 4568, 1244, 2136,
       3792, 1275, 3943], dtype=int32)

In [ ]:
# now comes the model creation